# Back Propagation Example 1

In [64]:
import numpy as np
from utils import *
from stratified import *
from neuralnetwork import *

### Forward Propagate

In [65]:
# Theta 1
# 0.40000  0.10000  
# 0.30000  0.20000  
theta1 = np.array([[0.4, 0.1],[0.3,0.2]])
# Theta 2 
#   0.70000  0.50000  0.60000
theta2 = np.array([0.7,0.5,0.6])
weightlist1= [theta1,theta2]

In [66]:
# Training set
# 	Training instance 1
# 		x: [0.13000]
# 		y: [0.90000]
# 	Training instance 2
# 		x: [0.42000]
# 		y: [0.23000]
# Training instance 1
trainingcategory = {'x1':'numerical', 'y':'class_numerical'}
trainingdata1 = np.array([0.13,0.9])
trainingdata2 = np.array([0.42,0.23])
inputdata1 = np.append(1,trainingdata1[0])
inputdata2 = np.append(1,trainingdata2[0])
exceptout1 = trainingdata1[1]
exceptout2 = trainingdata2[1]
lambda1 = 0

In [67]:
def forwardtest(inputdata,weightl,expectedout):
    current_layer_a = inputdata
    print('current_a at 1 is',current_layer_a)
    current_layer_index = 0
    alist = []
    alist.append(current_layer_a)
    for theta in weightl:
        z = np.dot(theta,current_layer_a)
        a = g(z)
        current_layer_a = np.append(1,a) if (current_layer_index+1 != len(weightl)) else a
        print('current_a at',current_layer_index+2,'is',current_layer_a)
        alist.append(current_layer_a)
        current_layer_index += 1
    result = current_layer_a
    print('prediction is', result)
    print('exceptout is', expectedout)
    print('cost is', costfunction(expectedout,result))
    return result, costfunction(expectedout,result), alist

In [68]:
r1,j1,a1 = forwardtest(inputdata1,weightlist1,exceptout1)
# Computing the error/cost, J, of the network
# 	Processing training instance 1
# 	Forward propagating the input [0.13000]
# 		a1: [1.00000   0.13000]

# 		z2: [0.41300   0.32600]
# 		a2: [1.00000   0.60181   0.58079]

# 		z3: [1.34937]
# 		a3: [0.79403]

# 		f(x): [0.79403]
# 	Predicted output for instance 1: [0.79403]
# 	Expected output for instance 1: [0.90000]
# 	Cost, J, associated with instance 1: 0.366

current_a at 1 is [1.   0.13]
current_a at 2 is [1.        0.601807  0.5807858]
current_a at 3 is 0.7940274264318581
prediction is 0.7940274264318581
exceptout is 0.9
cost is 0.36557477431084995
[array([1.  , 0.13]), array([1.       , 0.601807 , 0.5807858]), 0.7940274264318581]


In [69]:
r2,j2,a2 = forwardtest(inputdata2,weightlist1,exceptout2)
	# Processing training instance 2
	# Forward propagating the input [0.42000]
	# 	a1: [1.00000   0.42000]

	# 	z2: [0.44200   0.38400]
	# 	a2: [1.00000   0.60874   0.59484]

	# 	z3: [1.36127]
	# 	a3: [0.79597]

	# 	f(x): [0.79597]
	# Predicted output for instance 2: [0.79597]
	# Expected output for instance 2: [0.23000]
	# Cost, J, associated with instance 2: 1.276

current_a at 1 is [1.   0.42]
current_a at 2 is [1.         0.60873549 0.59483749]
current_a at 3 is 0.7959660671522611
prediction is 0.7959660671522611
exceptout is 0.23
cost is 1.2763768066887786


In [70]:
jlist1 = np.array([j1,j2])
numberofinstance1 = 2

In [71]:
def overallcost(jlist,n,weightl):
    s = sumofweights(weightl,bias=1)*lambda1/(2*n)
    jsum = np.sum(jlist)
    return jsum/n + s

In [72]:
overallcost(jlist1,numberofinstance1,weightlist1)

0.8209757904998143

### Back Propagate

In [75]:
def delta(weightl,a,expect,actual):
    delta_layer_n = actual-expect
    print('delta_layer_n is',delta_layer_n)

In [76]:
delta(weightlist1,a1,exceptout1,r1)

delta_layer_n is 0.10597257356814194


# Back Propagation Example 2